In [260]:
import pandas as pd
import numpy as np
import string
import csv
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from torchtext.data import BucketIterator, Field, TabularDataset
from torchtext import datasets
import torch
#from pycontractions import Contractions
from torchtext import data
import random
import spacy # this is a tokenizer
#spacy_en = spacy.load('en_core_web_sm') # this is a tokenizer (for english)
#print(os.listdir("../input"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gusmiriasa@GU.GU.SE/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/gusmiriasa@GU.GU.SE/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Dataset

In [261]:
"""
Getting to know our abusive/hate speech data
"""

'\nGetting to know our abusive/hate speech data\n'

In [262]:
all_data = pd.read_csv("labeled_data.csv") # raw csv with our data
# (for i in *.txt ; do cat $i ; echo ';...;' ; done ) >all_tweets.csv for unifying text files into csv

In [263]:
all_data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


## Preprocess Dataset

In [270]:
"""
Remove punctuation and decontract words
"""

'\nRemove punctuation and decontract words\n'

In [265]:
all_data['hate_speech'] = all_data['hate_speech'].astype('category')

In [266]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

all_data['clean_tweet'] = all_data['tweet'].apply(lambda x: remove_punct(x))
all_data.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,clean_tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldnt compl...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats coldtyga dwn bad for cuffin...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbabylife You ever f...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT CGAnderson vivabased she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you hear about me ...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",TMadisonx The shit just blows meclaim you so f...
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",BrighterDays I can not just sit up and HATE on...
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,selfiequeenbri cause Im tired of you big bitch...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",amp you might not get ya bitch back amp thats...
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",rhythmixx hobbies include fighting Mariam\n\n...


In [267]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

all_data['clean_tweet'] = all_data['clean_tweet'].apply(lambda x: decontracted(x))
all_data.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,clean_tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldnt compl...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats coldtyga dwn bad for cuffin...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbabylife You ever f...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT CGAnderson vivabased she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you hear about me ...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",TMadisonx The shit just blows meclaim you so f...
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",BrighterDays I can not just sit up and HATE on...
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,selfiequeenbri cause Im tired of you big bitch...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",amp you might not get ya bitch back amp thats...
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",rhythmixx hobbies include fighting Mariam\n\n...


In [268]:
df_two_columns = all_data[['hate_speech','clean_tweet']] # we extract label and tweet from csv for splitting into sets

In [269]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

train, test = train_test_split(df_two_columns,test_size=0.20)

train, val = train_test_split(train)

In [244]:
train_csv = pd.DataFrame(train) # making splits into new dataframes
test_csv = pd.DataFrame(test)
val_csv = pd.DataFrame(val)

In [245]:
train_csv.to_csv("train_preprocessed.csv") # writing and saving split dataframes into new csv files
test_csv.to_csv("test_preprocessed.csv")
val_csv.to_csv("valid_preprocessed.csv")

In [246]:
train_csv.tail(20) # checking what we are saving

,hate_speech,clean_tweet
18600,2,RT aychmoore most teachers abuse their positio...
5446,0,xpriceless CasioDatGuy bitch u got me fuckked up
8645,0,Danny Brown better put out more music before h...
6336,0,kanyewest tell yo bitch to stop calling me fam...
14982,0,RT DerrickJaxn I can not take any girl serious...
23021,0,Yall bitches FAKE
19261,0,RT iowahawkblog If you call a player a pussy w...
6467,1,kiefferjason nigga what I was talking bout aft...
3066,0,Dranumaya wrong maybe you been dealing with pu...
11465,0,Idgaf about you regardless bitch you got me al...


In [254]:
train_csv.shape # checking number of tweets in each splitted set

(14869, 2)

In [255]:
test_csv.tail(20)

,hate_speech,clean_tweet
19434,0,RT khulekile The karma of fucking over a good ...
5783,0,cullenbunn Maybe Baylee Ann Tater Pud Nurse Fe...
8599,0,Dad called me a border hopper
3147,0,EyeOnPolitics chrisloesch As long as its not o...
3245,0,GertrudeLarue \nI is colored Latino and some J...
9601,1,HOT ISIS SOCIAL MEDIA PHOTO Laughing Jihadis G...
19901,0,RT rebelliousmidy wheelerkashhh lol oh goddddd...
23078,0,Yall Niggahs got me fucked up thinkin ima soft...
17102,0,RT RiddMcCoy Where is everybody getting this f...
14293,0,RT BallOut stBlocJeremiah My Tryna make me go ...


In [204]:
test_csv.shape

(4957, 2)

In [256]:
val_csv.head(20)

,hate_speech,clean_tweet
15839,1,RT JoeBudden U famous niggas ruined the game b...
16504,0,RT Moniqve hoes be mad at you cause your nigga...
16885,0,RT Petahhdagod bitch httptcoGLUuGqzp
10346,0,I got a gang of old ass drafts all trash
11133,0,I would never fight a nigga for vaginathat is ...
11231,0,I am always dumb too a extra slow ass bitch
8506,0,Come home from work to Bridesmaids Twinkies an...
24651,0,we need to pile some hoes in the back of Ross car
21033,0,Some movies are made to facilitate dudes to pi...
10278,0,I feel bad for all you bitches that have never...


In [206]:
val_csv.shape

(4957, 2)

In [207]:
"""
TorchText Fields have a preprocessing argument. 
A function passed here will be applied to a sentence after it has been tokenized 
(transformed from a string into a list of tokens), but before it has been numericalized 
(transformed from a list of tokens to a list of indexes). 
This is where we'll pass our generate_bigrams function.
"""
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

## Split dataset into train, test and validation sets

In [208]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = generate_bigrams, include_lengths = True)
#LABEL = data.LabelField(dtype = torch.float) # not using torch.int because we expect a float between 0 and 1 for sentiment
LABEL = data.Field(use_vocab=False, sequential=False,dtype = torch.float)

In [209]:
fields = [('hate_speech', LABEL), ('clean_tweet', TEXT)] 

In [257]:
train_new_csv = pd.read_csv("train_preprocessed.csv") # checking that our newly splitted csv files look as expected
train_new_csv.head(30)

,Unnamed: 0,hate_speech,clean_tweet
0,21047,0,Some of yall gorgeous women ai not so gorgeous...
1,22250,0,Totaled RT CSharpCENT Crashed dat hoe RT viaNA...
2,9160,0,For some reason I always assume guys who act l...
3,23832,0,fuck ah bitch all I think about Is commas
4,13072,0,My mf dawggg I miss this hoe man httptcoNokkw...
5,3796,0,KenSuttling That twat Barry HearneIt would see...
6,20156,0,RT thecoreyholcomb When gay girls suck pussy f...
7,6620,0,madikat prolly not They sound like little bitches
8,9083,0,Females talk down on niccas when I can point o...
9,23782,0,dope in the bag pretty bitch on the side\nI se...


In [211]:
# splitting from one csv file
train_data, valid_data, test_data = TabularDataset.splits(path="", format='csv', train="train_preprocessed.csv", test="test_preprocessed.csv", validation="valid_preprocessed.csv", fields=fields, skip_header=True)

## Create vocabulary using pre-trained word embeddings

In [212]:
"""
Pre-trained vectors already have words with similar semantic meaning close together in vector space. 
This gives our embedding layer a good initialization as it does not have to learn these relations from scratch.
"""

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE)

LABEL.build_vocab(train_data)

print("Done building vocabulary.")

Done building vocabulary.


## Create the iterators, placing the tensors on the GPU if one is available

In [213]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), # these are the datasets
    batch_size = BATCH_SIZE,
    sort_key=lambda x:(len(x.clean_tweet), len(x.hate_speech)), # these are the tweets
    sort_within_batch = True, # In packed padded sequences al tensors within a batch need to be sorted by length
    device = device)

In [214]:
for i, batch in enumerate(train_iterator):
    trg = batch.hate_speech # target
    src = batch.clean_tweet # source
    
    print(trg)

tensor([2031., 4629., 1723., 6341., 8001., 6186., 6227., 5380., 1421., 4302.,
        3934., 9137., 3067., 3291., 5556., 2954., 8189., 2122., 9782., 4392.,
        1927., 8030., 9296., 7912., 9678., 8213., 1388., 8860., 2285., 7025.,
        2833., 2466., 1525., 7779., 9303., 4620., 3264., 8663., 1769., 2743.,
        3227., 1273., 7833., 9181., 5974., 6643., 3601., 4895., 5633., 9427.,
        6821., 7077., 2305., 2117., 1189., 9721., 8627., 7049., 9306., 3407.,
        5304., 8890., 4792., 9547., 6533., 7613., 7495., 1675., 8396., 9945.,
        7370., 7853., 4456., 5629., 7087., 8223., 9136., 8245., 8508., 8051.,
        3869., 3560., 9297., 4051., 4240., 5045., 2813., 3690., 8988., 1972.,
        8866., 7761., 8815., 6968., 3672., 1659., 1222., 8260., 6902., 6010.],
       device='cuda:0')
tensor([4252., 4769., 3589., 9930., 8585., 5426., 2543., 7939., 6151., 2863.,
        5180., 6285., 5882., 2173., 9299., 4952., 4594., 5279., 5120., 5344.,
        5386., 4714., 7757., 9582., 805

In [215]:
b = next(iter(train_iterator))
s, _ = b.clean_tweet
t = b.hate_speech

s.size(), t.size()
sample_source = s.transpose(1,0)[0].data.cpu().numpy()
#sample_target = t.transpose(1,0)[0].data.cpu().numpy()
print("source:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([TEXT.vocab.itos[o] for o in sample_source]), sample_source))
#print("target:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([TEXT.vocab.itos[o] for o in sample_target]), sample_target))

source:
0 

corresponding tensor:
[2] 



## Defining and building the model

In [216]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [217]:
"""
We'll create an instance of our RNN class, with the new parameters and arguments for the number of layers, 
bidirectionality and dropout probability. To ensure the pre-trained vectors can be loaded into the model, 
the EMBEDDING_DIM must be equal to that of the pre-trained GloVe vectors loaded earlier.
"""
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 500
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
#PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # pad token index from the vocabulary, getting the string representing the pad token from the field's pad_token
#print(PAD_IDX)
PAD_IDX = 0
model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [218]:
"""
Copying the pre-trained word embeddings we loaded earlier into the embedding layer of our model.
"""
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape) # checking if embeddings are the correct size (300 dimensions)

AttributeError: 'NoneType' object has no attribute 'shape'

In [219]:
"""
Replacing the initial weights of the embedding layer with the pre-trained embeddings.
"""
#model.embedding.weight.data.copy_(pretrained_embeddings) # should be done on the weight.data and not the weight

'\nReplacing the initial weights of the embedding layer with the pre-trained embeddings.\n'

In [220]:
"""
Since the <unk> and <pad> tokens aren't in the pre-trained vocabulary they have been initialized 
using unk_init when building our vocabulary. It is preferable to initialize them both to all zeros to explicitly 
tell our model that, initially, they are irrelevant for determining sentiment. We do this by manually setting 
their row in the embedding weights matrix to zeros. We get their row by finding the index of the tokens, 
which we have already done for the padding index.
"""
#UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
#print(UNK_IDX)
#model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# printing embedding weights matrix (two first rows are zeros but <pad> token will be learned)
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

## Training the model

In [221]:
import torch.optim as optim

# we use Adam because it gives parameters that are updated infrequently higher learning rates
optimizer = optim.Adam(model.parameters())

In [222]:
criterion = nn.BCEWithLogitsLoss()
# place the model and criterion on the GPU (if available)
model = model.to(device)
criterion = criterion.to(device)

In [223]:
"""
Calculating accuracy
"""
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [224]:
"""
As we have set include_lengths = True, our batch.text is now a tuple of shape: (numericalized tensor, actual lengths of each sequence) 
We separate these into their own variables, text and text_lengths, before passing them to the model.
"""
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() # use model.train() to ensure the dropout is "turned on" while training
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        clean_tweet, text_lengths = batch.clean_tweet
        
        predictions = model(clean_tweet, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.hate_speech)
        
        acc = binary_accuracy(predictions, batch.hate_speech)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [225]:
"""
Then we define a function for testing our model, again remembering to separate batch.text.
"""
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() # use model.eval() to ensure the dropout is "turned off" while evaluating
    
    with torch.no_grad(): # because we are evaluating
    
        for batch in iterator:

            clean_tweet, text_lengths = batch.clean_tweet
            
            predictions = model(clean_tweet, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.hate_speech)
            
            acc = binary_accuracy(predictions, batch.hate_speech)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [226]:
"""
How long the epochs take
"""
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [227]:
"""
The actual training here
"""
N_EPOCHS = 100

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'abusivelanguage-model.pt') # saving our model
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: -764436.874 | Train Acc: 0.01%
	 Val. Loss: -1445776.119 |  Val. Acc: 0.00%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: -2059796.216 | Train Acc: 0.01%
	 Val. Loss: -2709577.658 |  Val. Acc: 0.00%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: -3330897.442 | Train Acc: 0.01%
	 Val. Loss: -3990876.112 |  Val. Acc: 0.00%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: -4598205.676 | Train Acc: 0.01%
	 Val. Loss: -5262678.980 |  Val. Acc: 0.00%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: -5860300.785 | Train Acc: 0.01%
	 Val. Loss: -6538859.303 |  Val. Acc: 0.00%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: -7120253.877 | Train Acc: 0.01%
	 Val. Loss: -7815267.311 |  Val. Acc: 0.00%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: -8381290.389 | Train Acc: 0.01%
	 Val. Loss: -9069267.840 |  Val. Acc: 0.00%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: -9635592.223 | Train Acc: 0.01%
	 Val. Loss: -10331580.904 |  Val. Acc: 0.00%
Epoch: 09 | Epoch Time: 0m 1s
	Train Los

In [228]:
model.load_state_dict(torch.load('abusivelanguage-model.pt')) # loading our trained model

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: -126460195.630 | Test Acc: 0.00%


## Predictions

In [229]:
"""
Creating function to predict sentiment from sentence
"""
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [233]:
predict_sentiment(model, "terrible")

1.0